### Laod Search logs from CSV

In [0]:
df = spark.read.csv("/mnt/opas/opas-source/apt-roof-top-accuracy-improvement/rooftop_prod_logs_run_id_20251209_00032117.csv", header=True, inferSchema=True, sep=",", quote='"', escape='"')

usa_search_logs = df.filter("country == 'USA'")

print(usa_search_logs.count())
usa_search_logs.display()

### Load APT Dataset

In [0]:
from pyspark.sql.functions import col, expr

apt_dataset = spark.table("preprocess_prod.layer_14533")
apt_dataset = apt_dataset.filter(expr("exists(tags, x -> x.tagKey.key = 'metadata:country' AND x.value = 'USA')"))

# display(apt_dataset)
# DBTITLE 1


In [0]:
from pyspark.sql import functions as F

def convert_to_unsigned(layer_id, high, low):
    # layer_id = 14533
    unsigned_high = int(high) & ((1 << 64) - 1)
    unsigned_low = int(low) & ((1 << 64) - 1)
    return f"{layer_id}_{unsigned_high}_{unsigned_low}"


# Assuming df is your DataFrame
apt_Data_with_unsigned_id = apt_dataset.withColumn("unsigned_id", F.udf(convert_to_unsigned, "string")("id.layerId","id.high", "id.low"))

display(apt_Data_with_unsigned_id)

### Find missing search log APT Ids in layer data

In [0]:
from pyspark.sql.functions import col, explode

missing_ids_from_search_log = usa_search_logs.alias("search").join(apt_Data_with_unsigned_id.alias("layer"), col("search.provider_uid") == col("layer.unsigned_id"), "leftanti")
missing_ids_from_search_log.display()
print(missing_ids_from_search_log.count())


### Load APT Relocated CSV

In [0]:
relocated_apt_ds = spark.read.csv("/mnt/opas/opas-source/apt-roof-top-accuracy-improvement/relocated_usa_apt.csv", header=False, inferSchema=True, sep="|", quote='"', escape='"').withColumnRenamed("_c0", "singed_id") \
.withColumnRenamed("_c1", "location") \
.withColumnRenamed("_c2", "relocated_location") \
.withColumnRenamed("_c3", "relocated_bfp") \
.withColumnRenamed("_c4", "uuid") \
.withColumnRenamed("_c5", "comment") \
.withColumnRenamed("_c6", "hn_number") \
.withColumnRenamed("_c7", "street_name") \
.withColumnRenamed("_c8", "apt_id") 

display(relocated_apt_ds)